In [17]:
!wget https://raw.githubusercontent.com/fitomx19/ModelosEconometricos/master/Practica_3/uber_revisado2.csv /content

zsh:1: command not found: wget


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [19]:
# Step 1: Read the CSV file to load the data
uber_data = pd.read_csv('./uber_revisado2.csv', sep=',', encoding='utf-8')

In [20]:
uber_data

,Unnamed: 0,fare_amount,passenger_count,pickup_date,pickup_time,distancia
0,0,7.5,1,20150507,2,1.683323
1,1,7.7,1,20090717,2,2.457590
2,2,12.9,1,20090824,2,5.036377
3,3,5.3,3,20090626,0,1.661683
4,4,16.0,5,20140828,1,4.475450
...,...,...,...,...,...,...
190040,199995,3.0,1,20121028,0,0.112210
190041,199996,7.5,1,20140314,3,1.875050
190042,199997,30.9,2,20090629,3,12.850319
190043,199998,14.5,1,20150520,1,3.539715


In [21]:
# Step 2: Decide the features (X) and target variable (y)
# Assuming 'fare_amount' as the target variable and the rest as features
features = ['passenger_count',   'pickup_time', 'distancia']
X = uber_data[features]
y = uber_data['fare_amount']

# Step 3: Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Step 4: Standardize the features

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Return the shapes of the splits as a confirmation
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

((152036, 3), (38009, 3), (152036,), (38009,))

In [22]:
class ModelAgnosticFeatureImportance():
  def __init__(self,model,features,X_train,X_test,y_train,y_test) -> None:
    self.model = model
    self.features = features
    self.X_train = X_train
    self.X_test = X_test
    self.y_train = y_train
    self.y_test = y_test

  def evaluate(self):
    #Model performance with all features
    model = self.model.fit(self.X_train,self.y_train)
    base_performance = model.score(self.X_test,self.y_test)

    print(f"The Base Performance of the model given the metric is {base_performance}\nFeature\tPerformance\tDifference\n")

    #Model performance for every permutated feature
    for i, feature in enumerate(self.features):
      X_copy = np.copy(self.X_train)
      #Permutate feature
      np.random.shuffle(X_copy[:,i])
      #Training model
      permutated_model = self.model.fit(X_copy,self.y_train)
      #Testing
      feat_performance = permutated_model.score(self.X_test,self.y_test)

      print(f"{feature}\t{feat_performance}\t{base_performance-feat_performance}\n")

In [23]:
feature_importance = ModelAgnosticFeatureImportance(LinearRegression(),features,X_train_scaled,X_test_scaled,y_train,y_test)

In [24]:
feature_importance.evaluate()

The Base Performance of the model given the metric is 0.252348961155649
Feature	Performance	Difference

passenger_count	0.25217605668649146	0.0001729044691575199

pickup_time	0.25226820783229065	8.075332335832908e-05

distancia	0.0027539184145096707	0.2495950427411393

